# Disease Prediction

In [1]:
project_name = 'disease_prediction'

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [29]:
df = pd.read_csv('dataset.csv')
df = pd.read_csv('dataset.csv').where(pd.notnull(df), 'nan')

In [30]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Fungal infection,itching,skin_rash,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [31]:
df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Fungal infection,itching,skin_rash,dischromic _patches,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [32]:
def add_zero(*dictionary):
  for x in dictionary:
    if 1 in x:
        x[0] = 'nan'
    else:
      x['nan'] = 0

In [33]:
target_diseases = df.Disease.unique()
diseases_to_label = { value : key+1 for key,value in enumerate(target_diseases)}
label_to_diseases = {key+1: value for key,value in enumerate(target_diseases)}
symptoms = pd.unique(df[df.columns[1:]].values.ravel('K'))
symptoms_to_label = {value: key+1 for key,value in enumerate(symptoms)}
label_to_symptoms = {key+1: value for key,value in enumerate(symptoms)}
add_zero(diseases_to_label, symptoms_to_label, label_to_diseases, label_to_symptoms)

In [34]:
diseases_to_label, label_to_diseases, label_to_symptoms, symptoms_to_label

({'Fungal infection': 1,
  'Allergy': 2,
  'GERD': 3,
  'Chronic cholestasis': 4,
  'Drug Reaction': 5,
  'Peptic ulcer diseae': 6,
  'AIDS': 7,
  'Diabetes ': 8,
  'Gastroenteritis': 9,
  'Bronchial Asthma': 10,
  'Hypertension ': 11,
  'Migraine': 12,
  'Cervical spondylosis': 13,
  'Paralysis (brain hemorrhage)': 14,
  'Jaundice': 15,
  'Malaria': 16,
  'Chicken pox': 17,
  'Dengue': 18,
  'Typhoid': 19,
  'hepatitis A': 20,
  'Hepatitis B': 21,
  'Hepatitis C': 22,
  'Hepatitis D': 23,
  'Hepatitis E': 24,
  'Alcoholic hepatitis': 25,
  'Tuberculosis': 26,
  'Common Cold': 27,
  'Pneumonia': 28,
  'Dimorphic hemmorhoids(piles)': 29,
  'Heart attack': 30,
  'Varicose veins': 31,
  'Hypothyroidism': 32,
  'Hyperthyroidism': 33,
  'Hypoglycemia': 34,
  'Osteoarthristis': 35,
  'Arthritis': 36,
  '(vertigo) Paroymsal  Positional Vertigo': 37,
  'Acne': 38,
  'Urinary tract infection': 39,
  'Psoriasis': 40,
  'Impetigo': 41,
  'nan': 0},
 {1: 'Fungal infection',
  2: 'Allergy',
  3: 'G

In [35]:
for x in range(1,18):
    df[df.columns[x]] = df[df.columns[x]].apply(symptoms_to_label.get)
df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,2,35,56,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,2,35,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,35,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,2,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,2,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,7,15,43,69,60,92,0,0,0,0,0,0,0,0,0,0,0
4916,Acne,2,31,52,70,0,0,0,0,0,0,0,0,0,0,0,0,0
4917,Urinary tract infection,32,33,53,71,0,0,0,0,0,0,0,0,0,0,0,0,0
4918,Psoriasis,2,20,54,72,86,93,0,0,0,0,0,0,0,0,0,0,0


In [36]:
df[df.columns[0]] = df[df.columns[0]].apply(diseases_to_label.get)

In [44]:
df[1000:1050]

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
1000,19,19,7,11,34,43,22,44,59,113,118,0,0,0,0,0,0,0
1001,19,19,7,11,34,15,22,44,59,113,118,0,0,0,0,0,0,0
1002,19,19,7,11,34,15,43,44,59,113,118,0,0,0,0,0,0,0
1003,19,19,7,11,34,15,43,22,59,113,118,0,0,0,0,0,0,0
1004,19,19,7,11,34,15,43,22,44,113,118,0,0,0,0,0,0,0
1005,19,19,7,11,34,15,43,22,44,59,118,0,0,0,0,0,0,0
1006,19,19,7,11,34,15,43,22,44,59,113,0,0,0,0,0,0,0
1007,19,19,7,11,34,15,43,22,44,59,113,118,0,0,0,0,0,0
1008,19,19,7,11,34,15,43,22,44,59,113,118,0,0,0,0,0,0
1009,19,19,11,34,15,43,22,44,59,113,118,0,0,0,0,0,0,0


In [45]:
input_values = torch.tensor(df[df.columns[1:]].values, dtype=torch.float64)

In [46]:
df[df.columns[1:]]

,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,1,2,35,56,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,35,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,35,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,2,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,7,15,43,69,60,92,0,0,0,0,0,0,0,0,0,0,0
4916,2,31,52,70,0,0,0,0,0,0,0,0,0,0,0,0,0
4917,32,33,53,71,0,0,0,0,0,0,0,0,0,0,0,0,0
4918,2,20,54,72,86,93,0,0,0,0,0,0,0,0,0,0,0


In [47]:
output_values = torch.tensor(df[df.columns[0]].values)
input_values, output_values

(tensor([[ 1.,  2., 35.,  ...,  0.,  0.,  0.],
         [ 2., 35., 56.,  ...,  0.,  0.,  0.],
         [ 1., 35., 56.,  ...,  0.,  0.,  0.],
         ...,
         [32., 33., 53.,  ...,  0.,  0.,  0.],
         [ 2., 20., 54.,  ...,  0.,  0.,  0.],
         [ 2., 34., 55.,  ...,  0.,  0.,  0.]], dtype=torch.float64),
 tensor([ 1,  1,  1,  ..., 39, 40, 41]))

In [48]:
# input_values[:,torch.randperm(input_values.size()[1])]

## Prepare the Pytorch dataset, dataloader and train the model

In [70]:
dataset = TensorDataset(input_values, output_values)
train_dl = DataLoader(dataset, batch_size=125, shuffle=True)

In [80]:
for xb, yb in train_dl:
  print(xb.shape,yb.shape)
  break

torch.Size([125, 17]) torch.Size([125])


In [22]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [2]:
class ClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
    nn.Linear(17, 128),
    nn.ReLU(),
    nn.Linear(128,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.Linear(64,42))
      
  def forward(self, xb):
      out = self.network(xb.float())
      return out
  
  def training_step(self, batch):
      symptoms, labels = batch 
      symptoms = symptoms[:,torch.randperm(symptoms.size()[1])]
      out = self(symptoms)                  # Generate predictions
      loss = F.cross_entropy(out, labels) # Calculate loss
      return loss

  def validation_step(self, batch):
      symptoms, labels = batch
      symptoms = symptoms[:,torch.randperm(symptoms.size()[1])]
      out = self(symptoms)                    # Generate predictions
      loss = F.cross_entropy(out, labels)   # Calculate loss
      acc = accuracy(out, labels)           # Calculate accuracy
      return {'val_loss': loss, 'val_acc': acc}
      
  def validation_epoch_end(self, outputs):
      batch_losses = [x['val_loss'] for x in outputs]
      epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
      batch_accs = [x['val_acc'] for x in outputs]
      epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
      return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
  
  def epoch_end(self, epoch, result):
      print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(
          epoch, result['val_loss'], result['val_acc']))    
      
model = ClassifierModel()

In [106]:
def fit(epochs, lr, model, train_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, train_loader)
        if epoch%20==0:
          model.epoch_end(epoch, result)
        history.append(result)
    return history

In [107]:
history = fit(1000, 1e-2, model, train_dl)

Epoch [0], val_loss: 3.7091, val_acc: 0.0472
Epoch [20], val_loss: 2.6820, val_acc: 0.1355
Epoch [40], val_loss: 2.5616, val_acc: 0.1420
Epoch [60], val_loss: 2.2598, val_acc: 0.1746
Epoch [80], val_loss: 2.1905, val_acc: 0.1993
Epoch [100], val_loss: 2.2157, val_acc: 0.2336
Epoch [120], val_loss: 2.1573, val_acc: 0.2329
Epoch [140], val_loss: 2.4075, val_acc: 0.1937
Epoch [160], val_loss: 2.2527, val_acc: 0.1789
Epoch [180], val_loss: 2.0374, val_acc: 0.2503
Epoch [200], val_loss: 2.1560, val_acc: 0.2376
Epoch [220], val_loss: 2.1083, val_acc: 0.2162
Epoch [240], val_loss: 1.9298, val_acc: 0.2800
Epoch [260], val_loss: 1.9622, val_acc: 0.3063
Epoch [280], val_loss: 2.0458, val_acc: 0.2337
Epoch [300], val_loss: 1.7852, val_acc: 0.3592
Epoch [320], val_loss: 1.8606, val_acc: 0.3546
Epoch [340], val_loss: 1.8129, val_acc: 0.3348
Epoch [360], val_loss: 1.9424, val_acc: 0.2730
Epoch [380], val_loss: 1.8953, val_acc: 0.3445
Epoch [400], val_loss: 1.7689, val_acc: 0.3899
Epoch [420], val_lo

In [108]:
history2 = fit(1000, 2*1e-2, model, train_dl)

Epoch [0], val_loss: 3.8952, val_acc: 0.1181
Epoch [20], val_loss: 1.7420, val_acc: 0.3883
Epoch [40], val_loss: 1.4603, val_acc: 0.4670
Epoch [60], val_loss: 1.6295, val_acc: 0.4469
Epoch [80], val_loss: 1.5867, val_acc: 0.4481
Epoch [100], val_loss: 1.3036, val_acc: 0.5488
Epoch [120], val_loss: 1.3565, val_acc: 0.4896
Epoch [140], val_loss: 1.3880, val_acc: 0.4923
Epoch [160], val_loss: 1.7862, val_acc: 0.4294
Epoch [180], val_loss: 1.3528, val_acc: 0.4972
Epoch [200], val_loss: 1.4172, val_acc: 0.4853
Epoch [220], val_loss: 1.2204, val_acc: 0.5743
Epoch [240], val_loss: 1.1404, val_acc: 0.5802
Epoch [260], val_loss: 1.4727, val_acc: 0.4704
Epoch [280], val_loss: 1.6401, val_acc: 0.4277
Epoch [300], val_loss: 1.4453, val_acc: 0.4534
Epoch [320], val_loss: 1.2865, val_acc: 0.5258
Epoch [340], val_loss: 1.1612, val_acc: 0.5741
Epoch [360], val_loss: 1.1556, val_acc: 0.5771
Epoch [380], val_loss: 1.2174, val_acc: 0.5789
Epoch [400], val_loss: 1.2961, val_acc: 0.5656
Epoch [420], val_lo

In [109]:
history3 = fit(1000, 2*1e-4, model, train_dl)

Epoch [0], val_loss: 0.8240, val_acc: 0.7176
Epoch [20], val_loss: 0.6883, val_acc: 0.7799
Epoch [40], val_loss: 0.6913, val_acc: 0.7928
Epoch [60], val_loss: 0.6554, val_acc: 0.7942
Epoch [80], val_loss: 0.6578, val_acc: 0.7980
Epoch [100], val_loss: 0.6533, val_acc: 0.7990
Epoch [120], val_loss: 0.6420, val_acc: 0.8058
Epoch [140], val_loss: 0.6223, val_acc: 0.8187
Epoch [160], val_loss: 0.6268, val_acc: 0.8090
Epoch [180], val_loss: 0.6236, val_acc: 0.8042
Epoch [200], val_loss: 0.6173, val_acc: 0.8134
Epoch [220], val_loss: 0.6045, val_acc: 0.8242
Epoch [240], val_loss: 0.5971, val_acc: 0.8298
Epoch [260], val_loss: 0.6084, val_acc: 0.8153
Epoch [280], val_loss: 0.6011, val_acc: 0.8232
Epoch [300], val_loss: 0.6002, val_acc: 0.8214
Epoch [320], val_loss: 0.6127, val_acc: 0.8138
Epoch [340], val_loss: 0.5775, val_acc: 0.8326
Epoch [360], val_loss: 0.5921, val_acc: 0.8153
Epoch [380], val_loss: 0.5853, val_acc: 0.8337
Epoch [400], val_loss: 0.5852, val_acc: 0.8258
Epoch [420], val_lo

In [110]:
history4 = fit(100, 2*1e-5, model, train_dl)

Epoch [0], val_loss: 0.5315, val_acc: 0.8417
Epoch [20], val_loss: 0.5486, val_acc: 0.8374
Epoch [40], val_loss: 0.5304, val_acc: 0.8469
Epoch [60], val_loss: 0.5418, val_acc: 0.8427
Epoch [80], val_loss: 0.5385, val_acc: 0.8449


In [111]:
history3 = fit(100, 1e-6, model, train_dl)

Epoch [0], val_loss: 0.5369, val_acc: 0.8397
Epoch [20], val_loss: 0.5260, val_acc: 0.8471
Epoch [40], val_loss: 0.5418, val_acc: 0.8358
Epoch [60], val_loss: 0.5413, val_acc: 0.8422
Epoch [80], val_loss: 0.5306, val_acc: 0.8386


In [141]:
torch.save(model.state_dict(), 'classifier.pt')

In [9]:
sample = torch.tensor([0]*10+[1,2,35]+[0]*4)

In [50]:
help(torch.tensor)

Help on built-in function tensor:

tensor(...)
    tensor(data, dtype=None, device=None, requires_grad=False, pin_memory=False) -> Tensor
    
    Constructs a tensor with :attr:`data`.
    
    .. warning::
    
        :func:`torch.tensor` always copies :attr:`data`. If you have a Tensor
        ``data`` and want to avoid a copy, use :func:`torch.Tensor.requires_grad_`
        or :func:`torch.Tensor.detach`.
        If you have a NumPy ``ndarray`` and want to avoid a copy, use
        :func:`torch.as_tensor`.
    
    .. warning::
    
        When data is a tensor `x`, :func:`torch.tensor` reads out 'the data' from whatever it is passed,
        and constructs a leaf variable. Therefore ``torch.tensor(x)`` is equivalent to ``x.clone().detach()``
        and ``torch.tensor(x, requires_grad=True)`` is equivalent to ``x.clone().detach().requires_grad_(True)``.
        The equivalents using ``clone()`` and ``detach()`` are recommended.
    
    Args:
        data (array_like): Initial d

In [8]:
outputs= model(sample)#, predict(sample)
outputs

NameError: name 'sample' is not defined

In [61]:
def predict(symptoms):
   outputs= model(symptoms)
   _, preds = torch.max(outputs, dim=0)
   return label_to_diseases.get(int(preds))

In [53]:
predict(sample)

'Fungal infection'

In [54]:
for x in range(100):
  print(predict(input_values[x]))

Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Fungal infection
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
Allergy
GERD
GERD
GERD
GERD
GERD
GERD
GERD
GERD
GERD
GERD
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Chronic cholestasis
Osteoarthristis
Chronic cholestasis
Chronic cholestasis
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Allergy
Drug Reaction
Drug Reaction
Drug Reaction
Drug Reaction
Allergy
Peptic ulcer diseae
Peptic ulcer diseae
Peptic ulcer diseae
Peptic ulcer diseae
Peptic ulcer diseae
Peptic ulcer diseae
Hypertension 
Hypertension 
Peptic ulcer diseae
Peptic ulcer diseae
AIDS
Gastroenteritis
Gastroenteritis
Gastroenteritis
Heart attack
AIDS
AIDS
Gastroenteritis
Gastroenteritis
Heart attack
Diabetes 
Diabetes 
Diabetes 
Diabetes 
Diabetes 
Diabetes 
Diabete

In [55]:
accs = []
for x in range(10000):
  accs.append(int(accuracy(model(randomize(input_values)), output_values)*100))
sum(accs)/len(accs)

NameError: name 'accuracy' is not defined

In [4]:
model.load_state_dict(torch.load('classifier.pt'))

<All keys matched successfully>

In [57]:
randomize = lambda x: x[:,torch.randperm(x.size()[1])]

In [59]:
randomize(sample)

IndexError: tuple index out of range

In [7]:
torch.save(model,'model.pt')

/home/aakarshit/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type ClassifierModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [8]:
the_model = torch.load('model.pt')

In [9]:
the_model(sample)

tensor([  3.4492,  20.8962,  17.9403, -23.9720,  -5.0131,  16.4416,   3.7796,
         13.1577,  -2.8438,  15.7456,  -0.1325,   6.6301,  -7.2089,  -3.1740,
         15.0245,  -3.6137,   0.3588,  -8.5396,  -3.2298,  -2.8106,   1.3454,
         -1.4391,  -0.3175,   0.5536,  -4.3248,  -6.0544,   0.4294,  -3.2058,
         -9.5341,   4.3067,   2.7878,  -6.0709,   2.6026,  -8.9773,  -4.6979,
         -2.6365,  -3.2572,  -6.4231,  11.9692,   3.7777, -11.8342,   5.2651],
       grad_fn=<AddBackward0>)

In [2]:
class ClassifierModel_out(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
    nn.Linear(17, 128),
    nn.ReLU(),
    nn.Linear(128,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.Linear(64,42))
      
  def forward(self, xb):
      out = self.network(xb.float())
      return out

In [3]:
model = ClassifierModel_out()

In [4]:
model.load_state_dict(torch.load('classifier.pt'))

<All keys matched successfully>

In [6]:
torch.save(model,'model.pt')

In [10]:
traced_script_module = torch.jit.trace(model, sample)

In [11]:
traced_script_module.save("model.pt")